# Importing Libraries



In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image 
import matplotlib.pyplot as plt
import cv2 as cv2
import numpy as np
import pytesseract
import os
from google.colab import drive

import os

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Segment

In [ ]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def segment(image):
  import base64
  from roboflow import Roboflow
  import cv2
  import matplotlib.pyplot as plt
  import numpy as np

  image_read = cv2.imread(image)
  rf = Roboflow(api_key="EELutg6OW9WGerWim8P2")
  project = rf.workspace("monitor-detection").project("monitor-screen-detection-lcrfl")
  model = project.version(1).model
  image_read = cv2.resize(image_read, (1280,720),interpolation = cv2.INTER_CUBIC)
  # infer on a local image
  # model.predict(image).save("prediction.jpg") 
  result = model.predict(image).json()
  im_b64 = result['predictions'][0]['segmentation_mask']
  im_bytes = base64.b64decode(im_b64)
  im_arr = np.frombuffer(im_bytes, dtype=np.uint8)  # im_arr is one-dim Numpy array
  img = cv2.imdecode(im_arr, flags=cv2.IMREAD_UNCHANGED)
  img = cv2.resize(img, (1280,720),interpolation = cv2.INTER_CUBIC)
  # print(img.shape)
  pts = []
  found = 0
  for i in range(0,720):
    for j in range(0,1280):
      if img[i][j]==0:
        image_read[i][j] = 0
  # plt.subplot(141);plt.imshow(image_read,cmap="gray")
  # plt.subplot(142);plt.imshow(image_read)


  # plt.show()
  return image_read

# Cropping 

In [ ]:
def cropping(image,pts):
  import cv2
  import matplotlib.pyplot as plt
  import numpy as np
  width = 1280
  height = 720
  pts = np.float32(pts)
  converted_points = np.float32([[0,0],[width,0],[0,height],[width,height]])
  img = cv2.imread(image)
  matrix = cv2.getPerspectiveTransform(pts,converted_points)
  img_output = cv2.warpPerspective(img, matrix, (width,height))
  plt.imshow(img_output)
  plt.show()
  return 

# Classification

In [ ]:
import tensorflow as tf
import numpy as np
import keras.utils as image
from PIL import Image
from matplotlib import cm
def classify(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img = img.resize((256,256))
  img = np.array(img)
  test_img = np.expand_dims(img,axis=0)
  classifier = tf.keras.models.load_model('/content/drive/MyDrive/CloudPhy/Image_Classification.h5')
  cls=np.argmax(classifier.predict(test_img),axis=1)
  ans = cls[0]
  return ans

# Object Detection

In [ ]:
  import os 
  import glob as glob
  import matplotlib.pyplot as plt
  import cv2
  import requests
  import random
  import numpy as np

In [ ]:
def inference_obj(data_path):
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f"Current number of inference detection directories: {infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    # Inference on images.
    !python detect.py --weights runs/train/results_0/weights/best.pt \
    --source {data_path} --name {INFER_DIR} --save-crop
    return INFER_DIR

In [ ]:
import os

def getPropsFiles(path):
    file1 = []
    file2 = []
    props = f'{path}/'
    for f in os.listdir(props):
        f_path = os.path.join(props, f)
        if os.path.isfile(f_path):
            print(f)
        if "_W" in f_path:
            file1.append(f_path)
        else:
            file2.append(f_path)
    return file1, file2

# OCR Task

In [ ]:
import os
def ocr_task(arr):
  crops = []
  result = {}
  for x in arr:
    for file in os.listdir(f'{x}/'):
      img = cv2.imread(f'{x}/{file}')
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      blurred = cv2.GaussianBlur(gray, (5, 5), 0)
      kernel = np.array([[-1, -1, -1], 
                         [-1, 9, -1], 
                         [-1, -1, -1]]
                        )
      sharpened = cv2.filter2D(blurred, -1, kernel)
      im8 = 0
      im7 = 0
      try:
        im8 =int(pytesseract.image_to_string(sharpened, lang='eng', config='--psm 8 -c tessedit_char_whitelist=0123456789 '))
      except :
        im8 =0
      try:
        im7 = int(pytesseract.image_to_string(sharpened,lang='eng', config='--psm 7 -c tessedit_char_whitelist=0123456789 '))
      except:
        im7 = 0
      res = max(im7, im8)
      if "HR" in x:
        result["HR"] = res
      if "SBP" in x:
        result["SBP"] = res
      if "DBP" in x:
        result["DBP"] = res
      if "MAP" in x:
        result["MAP"] = res
      if "SPO" in x:
        result["SPO"] = res
      if "RR" in x:
        result["RR"] = res
  return result # dice containg value of heart rate, spo2 etc

# Brownie Point Task

In [ ]:
!pip install potracer
#image is a rgb image of the graph
def brownie(image):
  from potrace import Bitmap
  img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  ret,thresh1 = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
  t_lower = 50 
  t_upper = 150   
  edge = cv2.Canny(thresh1, t_lower, t_upper)
  bitmap = Bitmap(edge)
  path = bitmap.trace()  
  for curve in path:
    print ("Start Point of Curve:", curve.start_point)
    for segment in curve:
      print("Segment Point:",segment.end_point)
      if segment.is_corner:
          print("Corner Point:",segment.c)
      else:
          print("BezierSegment Start Point:",segment.c1)
          print("BezierSegment End Point:",segment.c2)

  return path  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Inference

In [ ]:
def inference(image_path:str): # dict
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  image_read_user = segment(image_path)
  cv2.imwrite('/content/drive/MyDrive/CloudPhy/infer/cropped.jpg',image_read_user)
  class_screen = classify(image_read_user)
  %ls
  %cd /content/drive/MyDrive/CloudPhy/yolov5_{class_screen}/
  !pwd
  !pip install -r requirements.txt
  IMAGE_INFER_DIR = inference_obj('/content/drive/MyDrive/CloudPhy/infer')
  PATH_CROPS = f'/content/drive/MyDrive/CloudPhy/yolov5_{class_screen}/runs/detect/{IMAGE_INFER_DIR}/crops'
  arr1, arr2 = getPropsFiles(PATH_CROPS)
  print(arr1)
  print(arr2)
  result = ocr_task(arr2)
  for x in arr1:
    for file in os.listdir(f'{x}/'):
      image_super = cv2.imread(f'{x}/{file}')
      brownie(image_super)
  ## put your code here
  
  return result

In [ ]:
result = inference('/content/drive/MyDrive/tridevdeoghar_icu_mon--4_2023_1_2_19_20_52.jpeg')

loading Roboflow workspace...
loading Roboflow project...
1/1 [==============================] - 0s 151ms/step
benchmarks.py    detect.py   __pycache__/      segment/        val.py
CITATION.cff     export.py   README.md         setup.cfg       yolov5m.pt
classify/        hubconf.py  README.zh-CN.md   train.py
CONTRIBUTING.md  LICENSE     requirements.txt  tutorial.ipynb
data/            models/     runs/             utils/
/content/drive/MyDrive/CloudPhy/yolov5_3
/content/drive/MyDrive/CloudPhy/yolov5_3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Current number of inference detection directories: 1
inference_2
detect: weights=['runs/train/results_0/weights/best.pt'], source=/content/drive/MyDrive/CloudPhy/infer, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=Fals

In [ ]:
print(result)

{'DBP': 60}
